In [1]:
from pyomo.environ import *
import random
import pandas as pd

In [16]:
dados = pd.read_csv("dados.csv", delimiter = ";", decimal = ",")
dados

,GR,Entrada,Parcela,Valor_Proposta,Inadimplencia
0,A,50,12,42071,0.001767
1,A,50,24,39829,0.005357
2,A,50,36,24569,0.019939
3,A,50,48,36311,0.022628
4,A,50,60,43397,0.041957
...,...,...,...,...,...
120,E,10,12,22814,0.174626
121,E,10,24,33351,0.125463
122,E,10,36,36179,0.304202
123,E,10,48,49671,0.126034


In [4]:
dados["GR"].unique()

array(['A', 'B', 'C', 'D', 'E'], dtype=object)

In [5]:
dados["Parcela"].unique()

array([12, 24, 36, 48, 60], dtype=int64)

In [6]:
dados["Entrada"].unique()

array([50, 40, 30, 20, 10], dtype=int64)

In [7]:
GrupoRisco = dados["GR"].unique() # array(['A', 'B', 'C', 'D', 'E'], dtype=object)
Parcelas = dados["Parcela"].unique() # array([12, 24, 36, 48, 60], dtype=int64)
Entrada = dados["Entrada"].unique() # array([50, 40, 30, 20, 10], dtype=int64)


listGrupoRisco = [i+1 for i in range(len(GrupoRisco))]
listParcelas = [i+1 for i in range(len(Parcelas))]
listEntrada = [i+1 for i in range(len(Entrada))]

deParaGrupoRisco = {GrupoRisco[i]:i+1 for i in range(len(GrupoRisco))}
deParaParcelas = {Parcelas[i]:i+1 for i in range(len(Parcelas))}
deParaEntrada = {Entrada[i]:i+1 for i in range(len(Entrada))}

In [9]:
paramValor = {}
paramInadimplencia = {}
for g in GrupoRisco:
    for e in Entrada:
        for p in Parcelas:
            paramValor[(deParaGrupoRisco[g],deParaEntrada[e],deParaParcelas[p])] = dados.loc[(dados.GR==g)&(dados.Entrada==e)&(dados.Parcela==p)].Valor_Proposta.iloc[0]
            paramInadimplencia[(deParaGrupoRisco[g],deParaEntrada[e],deParaParcelas[p])] = dados.loc[(dados.GR==g)&(dados.Entrada==e)&(dados.Parcela==p)].Inadimplencia.iloc[0]

In [10]:
ApetiteRisco = 0.03

model = ConcreteModel(name="credit")
model.setGrupoRisco = Set(initialize=listGrupoRisco)
model.setEntrada = Set(initialize=listEntrada)
model.setParcelas = Set(initialize=listParcelas)

model.politica = Var(model.setGrupoRisco, model.setEntrada, model.setParcelas, within=Binary, name="politica")

In [11]:
def limiteRisco(model):
    return quicksum(((ApetiteRisco - paramInadimplencia[(g,e,p)]) * paramValor[(g,e,p)]) * model.politica[(g, e, p)] 
             for g in model.setGrupoRisco for e in model.setEntrada for p in model.setParcelas) >= 0

model.limiteRisco = Constraint(rule = limiteRisco)

In [12]:
model.politicaAbertaGrupoRisco = ConstraintList()
for g in model.setGrupoRisco:
    for e in model.setEntrada:
        for p in model.setParcelas:
            if g > 1:
                model.politicaAbertaGrupoRisco.add(
                    model.politica[(g, e, p)] - model.politica[(g - 1, e, p)] <= 0
                )

model.politicaAbertaEntrada = ConstraintList()
for g in model.setGrupoRisco:
    for e in model.setEntrada:
        for p in model.setParcelas:
            if e > 1:
                model.politicaAbertaEntrada.add(
                    model.politica[(g, e, p)] - model.politica[(g, e - 1, p)] <= 0
                )

model.politicaAbertaParcelas = ConstraintList()
for g in model.setGrupoRisco:
    for e in model.setEntrada:
        for p in model.setParcelas:
            if p > 1:
                model.politicaAbertaParcelas.add(
                    model.politica[(g, e, p)] - model.politica[(g, e, p - 1)] <= 0
                )


In [13]:
model.Obj = Objective(
    expr=sum(paramValor[(g, e, p)] * model.politica[(g, e, p)]
             for g in model.setGrupoRisco for e in model.setEntrada for p in model.setParcelas),
    sense=maximize
)

In [26]:
#model.write('credit.lp')

solver = SolverFactory('gurobi')

results = solver.solve(model)
print(results)


Problem: 
- Name: x126
  Lower bound: 1438109.0
  Upper bound: 1438109.0
  Number of objectives: 1
  Number of constraints: 302
  Number of variables: 126
  Number of binary variables: 125
  Number of integer variables: 125
  Number of continuous variables: 1
  Number of nonzeros: 726
  Sense: maximize
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Wall time: 0.016925811767578125
  Error rc: 0
  Time: 0.15100765228271484
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [39]:
politica = {}
print("\nTOTAL APROVADO: ", model.Obj())
prod_pot_to

tal = 0
inad_pot_total = 0
prod_final = 0
inad_final = 0
print("---------------- Políticas")
for g in listGrupoRisco:
    print("Para o Grupo de Risco: ", GrupoRisco[g-1])
    if g not in politica: politica[g] = {}
    for p in listParcelas:
        if p not in politica[g]: politica[g][p] = {}
        minEntrada = 100
        for e in listEntrada:
            prod_pot_total += paramValor[(g, e, p)]
            inad_pot_total += paramValor[(g, e, p)] * paramInadimplencia[(g, e, p)]
            if value(model.politica[(g, e, p)].value) > 0.99:
                #print("--- Política ativa para [Grupo de Risco ", GrupoRisco[g-1], "][Entrada ", Entrada[e-1], "][Parcela ", Parcelas[p-1], "]")
                prod_final += paramValor[(g, e, p)]
                inad_final += paramValor[(g, e, p)] * paramInadimplencia[(g, e, p)]
                minEntrada = Entrada[e-1]
        politica[g][p] = minEntrada
        if minEntrada < 100: print("[", GrupoRisco[g-1],"][", Parcelas[p-1], "]: Entrada Mínima: ",minEntrada)
        else: print("[", GrupoRisco[g-1],"][", Parcelas[p-1], "]: Não Opera")
print("Valor Proposta Público Total: ", prod_pot_total)
print("% inad. Público Total: ", round(inad_pot_total / prod_pot_total*100, 2),"%")
print("Valor Aprovado Final: ", prod_final)
print("% inad. Público Final: ", round(inad_final / prod_final*100,2),"%")
print("% Valor Aprovado: ", round(prod_final / prod_pot_total*100,2),"%")


TOTAL APROVADO:  1438109.0
---------------- Políticas
Para o Grupo de Risco:  A
[ A ][ 12 ]: Entrada Mínima:  10
[ A ][ 24 ]: Entrada Mínima:  10
[ A ][ 36 ]: Entrada Mínima:  20
[ A ][ 48 ]: Entrada Mínima:  20
[ A ][ 60 ]: Entrada Mínima:  20
Para o Grupo de Risco:  B
[ B ][ 12 ]: Entrada Mínima:  10
[ B ][ 24 ]: Entrada Mínima:  40
[ B ][ 36 ]: Entrada Mínima:  50
[ B ][ 48 ]: Entrada Mínima:  50
[ B ][ 60 ]: Entrada Mínima:  50
Para o Grupo de Risco:  C
[ C ][ 12 ]: Entrada Mínima:  20
[ C ][ 24 ]: Entrada Mínima:  40
[ C ][ 36 ]: Não Opera
[ C ][ 48 ]: Não Opera
[ C ][ 60 ]: Não Opera
Para o Grupo de Risco:  D
[ D ][ 12 ]: Entrada Mínima:  30
[ D ][ 24 ]: Entrada Mínima:  50
[ D ][ 36 ]: Não Opera
[ D ][ 48 ]: Não Opera
[ D ][ 60 ]: Não Opera
Para o Grupo de Risco:  E
[ E ][ 12 ]: Entrada Mínima:  30
[ E ][ 24 ]: Entrada Mínima:  50
[ E ][ 36 ]: Não Opera
[ E ][ 48 ]: Não Opera
[ E ][ 60 ]: Não Opera
Valor Proposta Público Total:  3848176
% inad. Público Total:  11.18 %
Valor Apr

In [53]:
def modeloOtimizacaoPoliticaCredito(ApetiteRisco):

    model = ConcreteModel(name="credit")
    model.setGrupoRisco = Set(initialize=listGrupoRisco)
    model.setEntrada = Set(initialize=listEntrada)
    model.setParcelas = Set(initialize=listParcelas)

    model.politica = Var(model.setGrupoRisco, model.setEntrada, model.setParcelas, within=Binary, name="politica")
    
    def limiteRisco(model):
        return quicksum(((ApetiteRisco - paramInadimplencia[(g,e,p)]) * paramValor[(g,e,p)]) * model.politica[(g, e, p)] 
             for g in model.setGrupoRisco for e in model.setEntrada for p in model.setParcelas) >= 0

    model.limiteRisco = Constraint(rule = limiteRisco)
    
    model.politicaAbertaGrupoRisco = ConstraintList()
    for g in model.setGrupoRisco:
        for e in model.setEntrada:
            for p in model.setParcelas:
                if g > 1:
                    model.politicaAbertaGrupoRisco.add(
                        model.politica[(g, e, p)] - model.politica[(g - 1, e, p)] <= 0
                    )

    model.politicaAbertaEntrada = ConstraintList()
    for g in model.setGrupoRisco:
        for e in model.setEntrada:
            for p in model.setParcelas:
                if e > 1:
                    model.politicaAbertaEntrada.add(
                        model.politica[(g, e, p)] - model.politica[(g, e - 1, p)] <= 0
                    )

    model.politicaAbertaParcelas = ConstraintList()
    for g in model.setGrupoRisco:
        for e in model.setEntrada:
            for p in model.setParcelas:
                if p > 1:
                    model.politicaAbertaParcelas.add(
                        model.politica[(g, e, p)] - model.politica[(g, e, p - 1)] <= 0
                    )

    model.Obj = Objective(
        expr=sum(paramValor[(g, e, p)] * model.politica[(g, e, p)]
                 for g in model.setGrupoRisco for e in model.setEntrada for p in model.setParcelas),
        sense=maximize
    )
    
    solver = SolverFactory('gurobi')

    results = solver.solve(model)
    
    totalAprovado = model.Obj()
    
    politica = {}
    prod_pot_total = 0
    inad_pot_total = 0
    prod_final = 0
    inad_final = 0
    for g in listGrupoRisco:
        if g not in politica: politica[g] = {}
        for p in listParcelas:
            if p not in politica[g]: politica[g][p] = {}
            minEntrada = 100
            for e in listEntrada:
                prod_pot_total += paramValor[(g, e, p)]
                inad_pot_total += paramValor[(g, e, p)] * paramInadimplencia[(g, e, p)]
                if value(model.politica[(g, e, p)].value) > 0.99:
                    #print("--- Política ativa para [Grupo de Risco ", GrupoRisco[g-1], "][Entrada ", Entrada[e-1], "][Parcela ", Parcelas[p-1], "]")
                    prod_final += paramValor[(g, e, p)]
                    inad_final += paramValor[(g, e, p)] * paramInadimplencia[(g, e, p)]
                    minEntrada = Entrada[e-1]
            politica[g][p] = minEntrada

    
    
    return prod_final, inad_final / prod_final, prod_final / prod_pot_total


In [68]:
apetitesRisco = [round(i*0.01 + 0.01, 2) for i in range(12)]
apetitesRisco

[0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12]

In [69]:
taxaAprovacao = {}
valorAprovado = {}
inadAprovado = {}

for apetiteRisco in apetitesRisco:
    valorAprovado[apetiteRisco], inadAprovado[apetiteRisco], taxaAprovacao[apetiteRisco] = modeloOtimizacaoPoliticaCredito(apetiteRisco)

In [61]:
valorAprovado

{0.01: 310161,
 0.02: 915166,
 0.03: 1438109,
 0.04: 2026896,
 0.05: 2420855,
 0.06: 2777757,
 0.07: 3061122,
 0.08: 3311097,
 0.09: 3496876,
 0.1: 3681122,
 0.11: 3826054,
 0.12: 3848176}

In [62]:
inadAprovado

{0.01: 0.00994342688061368,
 0.02: 0.019938775254354862,
 0.03: 0.029967973502005443,
 0.04: 0.039594854194452855,
 0.05: 0.049728241526803885,
 0.06: 0.05994770889588339,
 0.07: 0.06977936394263934,
 0.08: 0.0798011113639766,
 0.09: 0.08993242731138834,
 0.1: 0.09845832515690031,
 0.11: 0.10669223520631715,
 0.12: 0.11178511253616154}

In [63]:
taxaAprovacao

{0.01: 0.08059948401528412,
 0.02: 0.23781812474273525,
 0.03: 0.3737118572539302,
 0.04: 0.5267160337780808,
 0.05: 0.6290915488272886,
 0.06: 0.7218373068175676,
 0.07: 0.7954734918569213,
 0.08: 0.8604328388306564,
 0.09: 0.9087099966321707,
 0.1: 0.9565887838809868,
 0.11: 0.9942513024352316,
 0.12: 1.0}